In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/gdrive')
%cd /content/drive/MyDrive/github/filter_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/github/filter_data


## Fill Placeholders



In [ ]:
!pip install indic-num2words

In [ ]:
import dataprep_utils
import importlib
importlib.reload(dataprep_utils)
lang_list = ['hi', 'en']

In [ ]:
# for lang in lang_list:
#   for i in range(10):
#     dataprep_utils.fill_placeholders(lang)

## Training

### Get manually created data

In [ ]:
import os
import pandas as pd

def get_training_data(lang_list):
    df = pd.DataFrame()
    # lang = 'en'
    all_data = pd.DataFrame()
    for lang in lang_list:
        read_dir = os.path.join(os.getcwd(),'formatted_data', lang)
        files = os.listdir(read_dir)
        data_files = [file for file in files if file.endswith(".csv")]

        for file in data_files:
            file_df = pd.read_csv(f'{os.path.join(read_dir, file)}')
            df = pd.concat([df, file_df], ignore_index=True)

    return df

In [ ]:
data = get_training_data(lang_list)
data = data.rename(columns={"sentence": "Text", "intent": "Label", 'language': 'lang'})
data.tail()

### Training utilities

In [ ]:
# !pip install transformers
# !pip install sentencepiece
# !pip install transformers[torch]
# # !pip install accelerate -U

In [ ]:
import argparse
import os
import pickle
import numpy as np
import pandas as pd
import torch

# import utils
# import importlib
# importlib.reload(utils)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)
torch.cuda.empty_cache()

MODEL = "ai4bharat/indic-bert"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_data(texts):
    tokenized_inputs = tokenizer(
        texts, padding="max_length", max_length=512, truncation=True
    )
    return tokenized_inputs


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="micro")
    precision = precision_score(y_true=labels, y_pred=pred, average="micro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="micro")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


def train(data):
    df = data
    df = df.dropna()

    labels = sorted(list(set(df["Label"])))
    labels_to_ids = {k: v for v, k in enumerate(sorted(labels))}

    X = list(df["Text"])
    y = list(df["Label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    X_train_tokenized = tokenizer(
        X_train, padding=True, truncation=True, max_length=512
    )
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL, num_labels=len(labels)
    )
    train_dataset = utils.Dataset(X_train_tokenized, y_train, labels_to_ids)
    val_dataset = utils.Dataset(X_val_tokenized, y_val, labels_to_ids)
    output_dir = os.path.join(os.getcwd(),'models')

    train_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="steps",
        eval_steps=500,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        seed=0,
        save_steps=500,
        load_best_model_at_end=True,
    )
    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
    with open(os.path.join(output_dir, "labels-dict.pkl"), "wb") as f:
        pickle.dump(labels_to_ids, f)

    train_metrics = trainer.train()
    eval_metrics = trainer.evaluate()

    print(train_metrics)
    print(eval_metrics)

## Train the model

In [ ]:
# train(data)

### Get GPT generated data

In [ ]:
lang_list = ['hi', 'en']
def get_gpt_data(lang_list):
  df = pd.read_csv(os.path.join(os.getcwd(),'gpt_data', 'all-lang-all.csv'))
  df = df[df['lang'].isin(lang_list)]
  return df

In [ ]:
gpt_data = get_gpt_data(lang_list)

In [ ]:
gpt_data.shape

(49499, 3)

In [ ]:
import pickle

import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
from utils import Dataset

MODEL = "ai4bharat/indic-bert"


def softmax(x):
    e_x = np.exp(x)
    return e_x / np.sum(e_x, axis=1, keepdims=True)


class IntentRecognizer:
    def __init__(self, model_path, label_dict_pkl, conf_threshold):
        with open(label_dict_pkl, "rb") as f:
            self.labels_to_ids = pickle.load(f)
        self.ids_to_labels = {
            intent_id: intent_label
            for intent_label, intent_id in self.labels_to_ids.items()
        }
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_path, num_labels=len(self.labels_to_ids)
        )
        self.test_trainer = Trainer(self.model)
        self.conf_threshold = conf_threshold

    def predict(self, sentence):
        sentence = [sentence]
        sentence_tokenized = self.tokenizer(
            sentence, padding=True, truncation=True, max_length=512
        )
        model_in = Dataset(sentence_tokenized)
        raw_pred, _, _ = self.test_trainer.predict(model_in)
        probs = softmax(raw_pred)
        y_pred = np.argmax(probs, axis=1)[0]
        pred_prob = np.max(probs)

        orig_pred = self.ids_to_labels[y_pred]
        # pred = self.ids_to_labels[y_pred]
        # if pred_prob < self.conf_threshold:
        #     pred = "unknown"
        return orig_pred, pred_prob

In [ ]:
import pandas as pd
# from intent_recognizer import IntentRecognizer
from sklearn.metrics import classification_report
from tqdm import tqdm


def test(data):
    df = data
    df = df.dropna()
    model_path = os.path.join(os.getcwd(),'models', 'jan25')

    intent_recognizer = IntentRecognizer(
        model_path, os.path.join(os.getcwd(),'models', 'labels-dict.pkl'), conf_threshold=0.85
    )

    true_intent = list()
    pred_intent = list()
    pred_probability = list()

    for i, row in tqdm(df.iterrows()):
        true_intent.append(row["Label"])
        intent, pred_prob = intent_recognizer.predict(row["Text"])
        pred_intent.append(intent)
        pred_probability.append(pred_prob)

    df["Predicted Intent"] = pred_intent
    df["Predicted Prob"] = pred_probability
    print(classification_report(true_intent, pred_intent))
    df.to_csv('gpt_data.csv', index=False)
    return df


In [ ]:
# filtered_data = test(gpt_data)